In [2]:
import pandas as pd
import numpy as np
import requests
import json
import openmeteo_requests
import requests_cache
from retry_requests import retry

In [3]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)
url = "https://archive-api.open-meteo.com/v1/archive"

In [67]:
months = ["OCAK 2024", "ŞUBAT 2024", "MART 2024", "NİSAN 2024", "MAYIS 2024", "HAZİRAN 2024"]
normal_columns = [("date","date"), ("kuzey", "breakfast"), ("kuzey", "lunch"), ("kuzey", "dinner"), ("guney", "breakfast"), ("guney", "lunch"), ("guney", "dinner"), ("hisar", "breakfast"), ("hisar", "lunch"), ("hisar", "dinner"), ("kilyos", "breakfast"), ("kilyos", "lunch"), ("kilyos", "dinner"),("kandilli", "breakfast"), ("kandilli", "lunch"), ("kandilli", "dinner"),("ahisari", "breakfast"), ("ahisari", "lunch"), ("ahisari", "dinner"),("cevrimdisi","breakfast"),("cevrimdisi","lunch"),("cevrimdisi","dinner"),("total","breakfast_total"),("total", "lunch_total"),("total", "dinner_total"),("total", "total")]
sahur_colums = [("date","date"),("kuzey", "sahur"), ("kuzey", "breakfast"), ("kuzey", "lunch"), ("kuzey", "dinner"), ("guney", "sahur"), ("guney", "breakfast"), ("guney", "lunch"), ("guney", "dinner"), ("hisar", "sahur"),  ("hisar", "breakfast"), ("hisar", "lunch"), ("hisar", "dinner"), ("kilyos", "sahur"), ("kilyos", "breakfast"), ("kilyos", "lunch"), ("kilyos", "dinner"),("kandilli", "sahur"), ("kandilli", "breakfast"), ("kandilli", "lunch"), ("kandilli", "dinner"),("ahisari", "sahur"), ("ahisari", "breakfast"), ("ahisari", "lunch"), ("ahisari", "dinner"),("cevrimdisi","sahur"), ("cevrimdisi","breakfast"),("cevrimdisi","lunch"),("cevrimdisi","dinner"),("total","sahur"),("total","breakfast_total"),("total", "lunch_total"),("total", "dinner_total"),("total", "total")]
meal_hours = [[8, 10], [12, 14], [17, 19]]
locations = {"kuzey":[41.0836, 29.0518], "guney":[41.0836, 29.0518], "hisar":[41.0892, 29.0509], "kilyos":[41.2446, 29.0128], "kandilli":[41.0638, 29.0653], "ahisari":[41.0836, 29.0518]}

In [68]:
sheets = []
for month in months:
    df = pd.read_excel("data.xlsx", sheet_name=month)
    sheets.append(df)

In [69]:
for i in range(len(sheets)):
    df = sheets[i]
    columns = normal_columns
    if months[i] in ["MART 2024", "NİSAN 2024"]:
        columns = sahur_colums
    df.columns = pd.MultiIndex.from_tuples(columns, names=["campus", "meal"])
    if months[i] in ["MART 2024", "NİSAN 2024"]:
        df.drop(columns=[("kuzey", "sahur"), ("guney", "sahur"), ("hisar", "sahur"), ("kilyos", "sahur"), ("kandilli", "sahur"), ("ahisari", "sahur"), ("cevrimdisi", "sahur"), ("total", "sahur")], axis=1, inplace=True)
    df.drop(columns = "total", axis=1, inplace=True)
    df.drop([0,1], inplace=True)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    df["date","date"] = pd.to_datetime(df.date.date).dt.strftime("%Y-%m-%d")

/var/folders/d0/4bp0h1hs3t96sr1kpxlzd6540000gn/T/ipykernel_58048/2134229131.py:9: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df.drop(columns = "total", axis=1, inplace=True)
/var/folders/d0/4bp0h1hs3t96sr1kpxlzd6540000gn/T/ipykernel_58048/2134229131.py:9: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df.drop(columns = "total", axis=1, inplace=True)
/var/folders/d0/4bp0h1hs3t96sr1kpxlzd6540000gn/T/ipykernel_58048/2134229131.py:9: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df.drop(columns = "total", axis=1, inplace=True)
/var/folders/d0/4bp0h1hs3t96sr1kpxlzd6540000gn/T/ipykernel_58048/2134229131.py:9: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df.drop(columns = "total", axis=1, inplace=True)
/var/folders/d0/4bp0h1hs3t96

In [70]:
df = pd.concat(sheets, axis=0, ignore_index=True)

In [71]:
df = df.melt(id_vars=[("date","date")])
df.columns = ["date", "campus", "meal", "count"]

In [72]:
kuzey_params = {
    "latitude": locations["kuzey"][0],
    "longitude": locations["kuzey"][1],
    "start_date": df['date'].min(),
    "end_date": df['date'].max(),
    "hourly": "temperature_2m"
}

guney_params = {
    "latitude": locations["guney"][0],
    "longitude": locations["guney"][1],
    "start_date": df['date'].min(),
    "end_date": df['date'].max(),
    "hourly": "temperature_2m"
}

hisar_params = {
    "latitude": locations["hisar"][0],
    "longitude": locations["hisar"][1],
    "start_date": df['date'].min(),
    "end_date": df['date'].max(),
    "hourly": "temperature_2m"
}

kilyos_params = {
    "latitude": locations["kilyos"][0],
    "longitude": locations["kilyos"][1],
    "start_date": df['date'].min(),
    "end_date": df['date'].max(),
    "hourly": "temperature_2m"
}

kandilli_params = {
    "latitude": locations["kandilli"][0],
    "longitude": locations["kandilli"][1],
    "start_date": df['date'].min(),
    "end_date": df['date'].max(),
    "hourly": "temperature_2m"
}

ahisari_params = {
    "latitude": locations["ahisari"][0],
    "longitude": locations["ahisari"][1],
    "start_date": df['date'].min(),
    "end_date": df['date'].max(),
    "hourly": "temperature_2m"
}

kuzey_response = openmeteo.weather_api(url, params=kuzey_params)
guney_response = openmeteo.weather_api(url, params=guney_params)
hisar_response = openmeteo.weather_api(url, params=hisar_params)
kilyos_response = openmeteo.weather_api(url, params=kilyos_params)
kandilli_response = openmeteo.weather_api(url, params=kandilli_params)
ahisari_response = openmeteo.weather_api(url, params=ahisari_params)

In [73]:
len(df)

3822

In [74]:
kuzey_weather = kuzey_response[0].Hourly().Variables(0).ValuesAsNumpy().reshape(-1, 24)
guney_weather = guney_response[0].Hourly().Variables(0).ValuesAsNumpy().reshape(-1, 24)
hisar_weather = hisar_response[0].Hourly().Variables(0).ValuesAsNumpy().reshape(-1, 24)
kilyos_weather = kilyos_response[0].Hourly().Variables(0).ValuesAsNumpy().reshape(-1, 24)
kandilli_weather = kandilli_response[0].Hourly().Variables(0).ValuesAsNumpy().reshape(-1, 24)
ahisari_weather = ahisari_response[0].Hourly().Variables(0).ValuesAsNumpy().reshape(-1, 24)

kuzey_breakfast_weather = kuzey_weather[:, meal_hours[0][0]:meal_hours[0][1]].mean(axis=1)
kuzey_lunch_weather = kuzey_weather[:, meal_hours[1][0]:meal_hours[1][1]].mean(axis=1)
kuzey_dinner_weather = kuzey_weather[:, meal_hours[2][0]:meal_hours[2][1]].mean(axis=1)

guney_breakfast_weather = guney_weather[:, meal_hours[0][0]:meal_hours[0][1]].mean(axis=1)
guney_lunch_weather = guney_weather[:, meal_hours[1][0]:meal_hours[1][1]].mean(axis=1)
guney_dinner_weather = guney_weather[:, meal_hours[2][0]:meal_hours[2][1]].mean(axis=1)

hisar_breakfast_weather = hisar_weather[:, meal_hours[0][0]:meal_hours[0][1]].mean(axis=1)
hisar_lunch_weather = hisar_weather[:, meal_hours[1][0]:meal_hours[1][1]].mean(axis=1)
hisar_dinner_weather = hisar_weather[:, meal_hours[2][0]:meal_hours[2][1]].mean(axis=1)

kilyos_breakfast_weather = kilyos_weather[:, meal_hours[0][0]:meal_hours[0][1]].mean(axis=1)
kilyos_lunch_weather = kilyos_weather[:, meal_hours[1][0]:meal_hours[1][1]].mean(axis=1)
kilyos_dinner_weather = kilyos_weather[:, meal_hours[2][0]:meal_hours[2][1]].mean(axis=1)

kandilli_breakfast_weather = kandilli_weather[:, meal_hours[0][0]:meal_hours[0][1]].mean(axis=1)
kandilli_lunch_weather = kandilli_weather[:, meal_hours[1][0]:meal_hours[1][1]].mean(axis=1)
kandilli_dinner_weather = kandilli_weather[:, meal_hours[2][0]:meal_hours[2][1]].mean(axis=1)

ahisari_breakfast_weather = ahisari_weather[:, meal_hours[0][0]:meal_hours[0][1]].mean(axis=1)
ahisari_lunch_weather = ahisari_weather[:, meal_hours[1][0]:meal_hours[1][1]].mean(axis=1)
ahisari_dinner_weather = ahisari_weather[:, meal_hours[2][0]:meal_hours[2][1]].mean(axis=1)

cevrimdisi_weather = (df['campus'] == "cevrimdisi").sum() * [np.nan]


In [75]:
a = np.concatenate([kuzey_breakfast_weather, kuzey_lunch_weather, kuzey_dinner_weather, guney_breakfast_weather, guney_lunch_weather, guney_dinner_weather, hisar_breakfast_weather, hisar_lunch_weather, hisar_dinner_weather, kilyos_breakfast_weather, kilyos_lunch_weather, kilyos_dinner_weather, kandilli_breakfast_weather, kandilli_lunch_weather, kandilli_dinner_weather, ahisari_breakfast_weather, ahisari_lunch_weather, ahisari_dinner_weather], dtype=np.float16)
x = a.round(2)
weather = np.concatenate([x, cevrimdisi_weather], dtype=np.float16)

In [76]:
df.insert(3, "weather", weather)

In [77]:
df.to_csv("data.csv", index=False)